# Welcome to UBLyter !

_UBLyter_ is an [everis](https://www.everis.com/global/en) **SEMBU** environment for the generation of [OASIS UBL-2.x](http://docs.oasis-open.org/ubl/cs01-UBL-2.2/UBL-2.2.html) artefacts. The current version is UBLyter v0.1, which focuses on the production of one type of model (differently to the staged approach used in UBL for eProcurement, transport, etc. Thus no "_stage_" is used herein, but "_model_"). 

The status of the development is "just started!". Feel free to contribute by creating issues and commenting on the [Ublyter GitHub](https://github.com/everis-sembu/ublyter).

## Edit the model

* Download the [UBL-2.3 model](./mod/UBL-Pre-award-2.3.ods),

* Modify the model (introduce the changes you need),

* Upload it ([how to upload](http://everis-sembu.github.io/how-to-upload.html))

## Set the main distribution parameters

In [69]:
String 
    modelName      = "Pre-award",
    datetimelocal  = "20190331-1500z",
    UBLversion     = "2.3", // The version of the UBL distribution this model is based on.
    UBLprevVersion = "2.2", // The previous version of UBL. Backwards compatibility is checked.
    ABIEextensions = "yes"; // Override this as "no" for UBL 2.2 and earlier.

## Environment configuration

### Location of the `*.jar` files

Jupyter-notebook needs to know where to locate the saxon and other java resources. There are several ways of doing this:

1. From the Java code. However trying to modify the environment from the Java execution is complicated and prone to nasty effects. Nonetheless it is possible, see this thread for alternatives: https://stackoverflow.com/questions/318239/how-do-i-set-environment-variables-from-java; or
2. From the OS environment configuration: e.g. in Linux you can set the variables in your `~/.profile` or your `~/.bashrc` files; or
3. In the terminal you'll use to run the Jupyter notebook, before running Jupyter, key in: `export IJAVA_CLASSPATH=./lib/*.jar`.

### Cranesoftwright's tooling specification

The core of the work during the generation of the UBL artefacts is done via XSL-T transformations. The stylesheets used for these transformations were produced by [Cranesoftwrights](https://www.cranesoftwrights.com/htdocs/) (Ken Holman).

The following variables are used to identify which version of the Cranesoftwrights tools are being used for the transformations. These timestamps are also used in the names of the folders where these resources are located. See the content of the directory `./util`.

In [70]:
// Cranesoftwrights tooling timestamps. Could be moved onto a .properties file
String 
    gc2obdndr = "20190320-0140z",
    ods2obdgc = "20180921-2010z",
    cvagcxsl  = "20130416-0040z",
    cva2sch   = "20130207-1940z",
    gc2odsxml = "20170727-0220z";

### Skippers

Transformations are costly. If a transformation has generated successfully its output there is no sense in wasting time re-executing that transformation once and again. 

The following variables can be used to skip successful transformations at debugging time. Set the "*_Skip" variable to `true` if you do not want a transformation to be executed. 

Beware that a full execution of the whole notebook requires that all the `*Skip` variables are set to `false`.

In [71]:
boolean rawCopy_Skip          = true; // Skips having to copy the raw directory files more than once.
boolean ods2gc_Signature_Skip = true; // Skips the transformation of the Signature ODS model into its pivot GC file.
boolean ods2gc_Entities_Skip  = true; // Skips the transformation of the UBL ccts model into its corresponding pivot GC file.   
boolean ods2gc_Checks_Skip    = true; // Skips checking the backwards compatibility and the NDR compliance.
boolean gc2ndr_Skip           = true;// Skips the generation of the XSDs.    

### Global directory and file path names

The following variables are used along the lifecycle of the execution to specify input and output directory and file names.

In [72]:
// Input folders and artefacts. All variables herein could be moved onto a .properties file.
String inDir      = "./input/"; // The directory where pre-existing artefacts necessary are located.
String inMod      = inDir + "mod/"; // Location of the model. In UBL models are maintained as spread-sheets!
String inIdent    = inDir + "ident/"; // Metadata for the identification of the BIE(s), namespaces, UBLversion, etc.
String inMassage  = inDir + "massage/"; // ODS tabs names need to be spanned.
String inGCDir    = inDir + "gc/"; // Location of input gc files, e.g. previous version of the UBL-Entitites as gc files.
// Common path names
String utilDir                 = "./util/"; // Utilities directory: Cranesoftwright's XSL-T files.
String lengthen_model_name_uri = inMassage + "massageModelName-" + UBLversion + ".xml";
String configDir               = "./conf/"; // The Model configuration file folder.
String modelConfFile           = configDir + "config-UBL-" + UBLversion + "-" + modelName + ".xml";
String spellFile               = utilDir + "wordlist-UBL-" + UBLversion + ".txt"; // To check the spelling of the words in the model.
// Cranesoftwright's transformation stylesheet
String ods2gcXSL  = utilDir + "Crane-ods2obdgc-" + ods2obdgc + "/Crane-ods2obdgc.xsl";
String checkXSL   = utilDir + "Crane-gc2obdndr-" + gc2obdndr + "/Crane-checkgc4obdndr.xsl";
String gc2ndrXSL  = utilDir + "Crane-gc2obdndr-" + gc2obdndr + "/Crane-gc2obdndr.xsl";
// Output folders and artefacts
String outDir     = "./output/"; // The directory where the newly generated artefacts are put.
String outGCDir   = outDir + "gc/";
String distDir    = outDir + modelName + "-" + UBLversion + "/"; // The directory where the distribution artefacts are put.
String outHTMLDir = outDir + "html/";
String outDBDir   = outDir + "db/"; // DocBook xml files directory
String outJunkDir = outDir + "tmp/"; // Temporary trash;
// Common path names
String utilDir                 = "./util/"; // Utilities directory: Cranesoftwright's XSL-T files.
String lengthen_model_name_uri = inMassage + "massageModelName-" + UBLversion + ".xml";
String configDir               = "./conf/"; // The Model configuration file folder.
String modelConfFile           = configDir + "config-UBL-" + UBLversion + "-" + modelName + ".xml";
String spellFile               = utilDir + "wordlist-UBL-" + UBLversion + ".txt"; // To check the spelling of the words in the model.
String UBLCommonLibrary         = "CommonLibrary"; // In UBL-2.3 GC files the model name column is named like this, differently to UBL-2.2

## Raw files

The UBL distribution is organised around a bunch of **constant** folders and files that are part of the data architecture, data validation, documentation, etc. There is no point in creating these from scratch every time, so they are copied from a kind of template (*raw*) directory into the target output directory.

This implies that the "raw" folder, and its content, is necessary **before** the production of the XSD schemas. So make sure that the **_raw_** folder exists and contains the right original files. 

The next cells are used to:

1. Specify where these "raw" folder and files are located, and
2. Copy them in the target (output) directory.

In [73]:
String rawDir = inDir + "raw" + "/";

In [74]:
import org.apache.commons.io.FileUtils;
import java.io.File;
import java.io.IOException;

// Remember to set the skipper variable to false if you need to copy again the raw directory.
if(!rawCopy_Skip){
    try {
        FileUtils.copyDirectory(new File(rawDir), new File(distDir));
    } catch (IOException e) {
        e.printStackTrace();
    }
}

## Creation of the GeneriCode (GC) files

[GeneriCode](http://docs.oasis-open.org/codelist/ns/genericode/1.0/) (GC) is an OASIS XML specification for the expression of tables. The OASIS UBL tooling uses GC as a **_pivot_** format that is used in several different ways, e.g. to check NDR and as the input for subsequent generation of W3C XSD schemas, [CVA](http://docs.oasis-open.org/codelist/cs01-ContextValueAssociation-1.0/doc/context-value-association.html) files, HTML reporting and other types of transformations.

### Configuration

#### Common variables needed during the transformation

In [75]:
// ODS tab names are short and need to be spanned conveniently
String lengthen_model_name_uri  = inMassage + "massageModelName-" + UBLversion + ".xml";
// The XSL-T base directory is "./util/Crane-ods2obdgc-20180921-2010z" (whilst the jupyter notebook is "./") hence the ident file is in "../../ident/"
String identification_from_util_folder = "../../" + inDir + "ident/"; 
// Common parameter for the path building
String cit                      = "-it:ods-uri";

### Generation of the GC files

The method below launches the actual transformation of the ods model into the pivot Genericode file. Depending of your machine features, the jupyter notebook environment may need up to more than 15 minutes. So take it easy.

#### Reusable methods for the transformation

In [76]:
import java.util.Calendar;
import net.sf.saxon.Transform;
import java.text.SimpleDateFormat;

public void t(String[] argList){
    Transform.main(argList);
}

public void showStartTime(){
    Calendar cal = Calendar.getInstance();
    SimpleDateFormat sdf = new SimpleDateFormat("HH:mm:ss");        
    System.out.println("Transformation started at: " + sdf.format(cal.getTime()) + " this may take some minutes...");
}
public void showEndTime(){
    Calendar cal = Calendar.getInstance();
    SimpleDateFormat sdf = new SimpleDateFormat("HH:mm:ss");        
    System.out.println("Transformation ended at: " + sdf.format(cal.getTime()));
}

#### Variables for the UBL signature components

In [77]:
// Input files for the generation of the UBL signature components as a GC file

String UBLsignatureSource              = inMod + "UBL-Signature-" + UBLversion + ".ods";
String UBLsignatureIdentification      = inIdent + "ident-UBL-Signature-"+ UBLversion + "-" + modelName + ".xml";

//UBL signature GC output file
String UBLsignatureTarget = outGCDir + "UBL-Signature-" + UBLversion + ".gc";

// Parameters for the generation of the UBL signature components
String UBLsignatureXSL       = "-xsl:" + ods2gcXSL;
String UBLsignatureO         = "-o:" + UBLsignatureTarget;
String UBLsignatureODS_uri   = "ods-uri=" + UBLsignatureSource;
String UBLsignatureIdent_uri = "identification-uri=" + identification_from_util_folder + "ident-UBL-Signature-"+ UBLversion + "-" + modelName + ".xml";

#### Check whether the input files exist

In [78]:
if(!new File(ods2gcXSL).exists()) System.err.println("File " + ods2gcXSL + " was not located.");
if(!new File(UBLsignatureSource).exists()) System.err.println("File " + UBLsignatureSource + " was not located in folder " + inMod);
if(!new File(UBLsignatureIdentification).exists()) System.err.println("File " + UBLsignatureIdentification + " was not located in folder " + inIdent);

#### Launch the transformation of the UBL signature components

In [79]:
// Remember to set the skipper variable to `false` if you want this code to be executed!
if (!ods2gc_Signature_Skip){
    showStartTime();
    String[] argsLine = {ods2gcXSL, UBLsignatureO, cit, UBLsignatureODS_uri, UBLsignatureIdent_uri};
    t(argsLine);
    showEndTime();
}

#### Variables for the UBL library components 

In [80]:
// Input files for the generation of the UBL ccts-based entities as a GC file
String UBLentitiesSource1              = inMod + "UBL-Library-" + UBLversion + ".ods";
String UBLentitiesSource2              = inMod + "UBL-Documents-" + UBLversion + ".ods";
String UBLentitiesIdentification       = inIdent + "ident-UBL-"+ UBLversion + "-" + modelName + ".xml";

//UBL entities GC output file
String UBLentitiesTarget  = outGCDir + "UBL-Entities-" + UBLversion + ".gc";

// Parameters for the generation of the UBL entities:
String UBLentitiesO         = "-o:" + UBLentitiesTarget;
String UBLentitiesODS_uri   = "ods-uri=" + UBLentitiesSource1 + "," + UBLentitiesSource2;
String UBLentitiesIdent_uri = "identification-uri=" + identification_from_util_folder + "ident-UBL-"+ UBLversion + "-" + modelName + ".xml";


#### Check whether the input files exist

In [81]:
if(!new File(UBLentitiesSource1).exists()) System.err.println("File " + UBLentitiesSource1 + " was not located in folder " + inMod);
if(!new File(UBLentitiesSource2).exists()) System.err.println("File " + UBLentitiesSource2 + " was not located in folder " + inMod);
if(!new File(UBLentitiesIdentification).exists()) System.err.println("File " + UBLentitiesIdentification + " was not located in folder " + inIdent);
if(!new File(lengthen_model_name_uri).exists()) System.err.println("File " + lengthen_model_name_uri + " was not located in folder " + inMassage);

#### Launch the transformation into a GC file

In [82]:
// Remember to set the skipper variable to `false` if you want this code to be executed!
if (!ods2gc_Signature_Skip){
    showStartTime();
    String[] argsLine = {ods2gcXSL, UBLentitiesO, cit, UBLentitiesODS_uri, UBLentitiesIdent_uri};
    t(argsLine);
    showEndTime();
}

## Generation of the XSD Schemas

The generation of the XSD Schemas starts here. It takes as inputs the UBL entities and signature GC files generated in the previous steps.

This phase is divided into several steps:

1. Before launching the creation of the XSD schemas, a comparison between the previous and the current UBL versions is performed to check the backwards compatibility;
2. A strong validation of the model against the OASIS UBL Naming and Design Rules ([NDR](http://docs.oasis-open.org/ubl/Business-Document-NDR/v1.0/Business-Document-NDR-v1.0.html)) -now in the GC files-, is performed;
3. An [HTML report](http://localhost:8888/view/output/html/check-ubl-2.3-ubl-2.2.html) is created containing the results of the two previous steps;
4. The creation of the XSD schemas is launched.


### Backwards compatibility and NDR conformance checks

#### Setting the parameters for the transformation

In [83]:
// Input and output files.
String checkSrc                 = UBLentitiesTarget; // PIPELINE: The output of the previous phase is the input of this phase! 
String checkOldURI              = inGCDir + "UBL-Entities-" + UBLprevVersion + ".gc";
String checkDBCmnURI            = outDBDir + "old2newDocBook-UBL-" + UBLversion + "-" + modelName + "-UBL-" + UBLprevVersion + "-library.xml";
String checkDBMainURI           = outDBDir + "old2newDocBook-UBL-" + UBLversion + "-" + modelName + "-UBL-" + UBLprevVersion + "-documents.xml";
String checkgc4obdndrReport     = outHTMLDir + "check-ubl-" + UBLversion + "-ubl-" + UBLprevVersion + ".html";
String checkgc4obdndrAllReports = outHTMLDir + "All-UBL-" + UBLversion + "-Documents.html";
String checkgc4obdndrConfFile   = "../../" + modelConfFile;

// Parameters for the transformation.
String checkgc4obdndrXSL        = "-xsl:" + checkXSL;
String checkgc4obdndrO          = "-o:" + checkgc4obdndrReport;
String checkgc4obdndrSource     = "-s:" + checkSrc;
String checkgc4obdndrConfig_uri = "config-uri=" + checkgc4obdndrConfFile;
String checkgc4obdndrTitle      = "title-suffix=" + "Universal Business Language (UBL) "+ UBLversion + " " + modelName + " to " + UBLprevVersion;
String checkgc4obdndrChange     = "change-suffix=" + "UBL";
String checkParamOldURI         = "../../" + checkOldURI; // Base directory for the execution of the XSL-T.
String checkgc4obdndrCompare    = "old-uri=" + checkParamOldURI;
String checkgc4obdndrCmnURI     = "docbook-common-uri=" + checkDBCmnURI;
String checkgc4obdndrMainURI    = "docbook-maindoc-uri=" + checkDBMainURI;
String checkgc4obdndrDBHRef     = "all-documents-report-href=" + checkgc4obdndrAllReports;
String checkgc4obdndrWords      = "den-word-list-uri=" + spellFile;
String checkgc4obdndrUBLlibName = "common-library-singleton-model-name=" + UBLCommonLibrary;
String checkgc4obdndrCurVersion = "version-column-name=CurrentVersion";
String checkgc4obdndrSuprXMLns  = "--suppressXsltNamespaceCheck:on";

#### Check input files

In [84]:
if(!new File(modelConfFile).exists()) System.err.println("File " + modelConfFile + " was not located.");
if(!new File(checkXSL).exists()) System.err.println("File " + checkXSL + " was not located.");
if(!new File(checkSrc).exists()) System.err.println("File " + checkSrc + " was not located.");
if(!new File(checkOldURI).exists()) System.err.println("File " + checkOldURI + " was not located.");

#### Launch the checks (backwards compatibility and NDR compliance)

In [85]:
import java.text.SimpleDateFormat;
import java.util.Calendar;

// TODO
public void showReport(){

}

// Remember to set the skipper variable to `false` if you want this code to be executed!
if (!ods2gc_Checks_Skip){

    try{
        showStartTime();

        String[] argList = {
                checkgc4obdndrXSL, 
                checkgc4obdndrSource,
                checkgc4obdndrO, 
                checkgc4obdndrConfig_uri, 
                checkgc4obdndrTitle, 
                checkgc4obdndrChange,
                checkgc4obdndrCompare, 
                checkgc4obdndrCmnURI,
                checkgc4obdndrMainURI,
                checkgc4obdndrDBHRef,
                checkgc4obdndrWords,
                checkgc4obdndrUBLlibName,
                checkgc4obdndrCurVersion,
                checkgc4obdndrSuprXMLns};

        t(argList); // See reusable Transform methods above.

        showEndTime();
        showReport();
        
        }catch(Exception e){
            e.printStackTrace();
        }
}

Transformation started at: 21:54:42 this may take some minutes...
Transformation ended at: 21:54:53


#### Backwards and NDR-compliance report

If the execution of the previous cell did not cast any error the report should be accessible [here](http://localhost:8888/view/output/html/check-ubl-2.3-ubl-2.2.html).


### XSD Generation

#### Setting the parameters for the transformation

In [86]:
// Input and output files.
String gc2ndrSrc                = UBLentitiesTarget; // PIPELINE: The output of the previous phase is the input of this phase! 
String gc2ndrJunk               = outJunkDir + modelName + "-" + UBLversion + "-junk.out";
String gc2ndrConfFile           = "../../" + modelConfFile; // XSL has a different execution base directory than jupyter.
String gc2ndrErrorsAreFatal     = "yes";

// Parameters for the transformation.
String gc2ndrParXSL             = "-xsl:" + gc2ndrXSL;
String gc2ndrSource             = "-s:" + gc2ndrSrc;
String gc2ndrO                  = "-o:" + gc2ndrJunk;
String gc2ndrConfig_uri         = "config-uri=" + gc2ndrConfFile;
String gc2ndrErrorBehaviour     = "errors-are-fatal=" + gc2ndrErrorsAreFatal;  
String gc2ndrSuprXMLns          = "--suppressXsltNamespaceCheck:on";
String gc2ndrABIEext            = "extensions-for-abies=" + ABIEextensions;

#### Check input files

In [87]:
if(!new File(gc2ndrXSL).exists()) System.err.println("File " + gc2ndrXSL + " was not located.");
if(!new File(gc2ndrSrc).exists()) System.err.println("File " + gc2ndrSrc + " was not located.");
if(!new File(modelConfFile).exists()) System.err.println("File " + modelConfFile + " was not located.");

#### Launch the generation of XSDs

In [88]:
// Remember to set the skipper variable to `false` if you want this code to be executed!
if (!gc2ndr_Skip){
    showStartTime();
    String[] argsLine = {gc2ndrParXSL, gc2ndrSource, gc2ndrO, gc2ndrConfig_uri, gc2ndrErrorBehaviour, gc2ndrSuprXMLns, gc2ndrABIEext};
    t(argsLine);
    showEndTime();
}